In [1]:
import pandas as pd

In [10]:
# Reading in BERT confidences and predictions for training set 2
train2_BERT_confidence = pd.read_csv("/content/drive/MyDrive/UC Berkeley MIDS/W266/t5_dataset/BERT_Confidence_Predictions_80k_Stage_2.csv", sep = ",", header=0)
train2_BERT_preds = pd.read_csv("/content/drive/MyDrive/UC Berkeley MIDS/W266/t5_dataset/BERT_Predictions_80k_stage_2_1_to_5.csv", sep = ",", header=0)
# Merging BERT predictions and confidences for training set 2
train2_BERT = pd.merge(train2_BERT_preds,train2_BERT_confidence,on = ["Unnamed: 0","Unnamed: 0"])

In [3]:
# Reading in BERT confidence and predictions test sets
test_BERT_confidence = pd.read_csv("/content/drive/MyDrive/UC Berkeley MIDS/W266/t5_dataset/BERT_X_Test_Confidence_Predictions.csv", sep = ",", header=0)
test_BERT_preds = pd.read_csv("/content/drive/MyDrive/UC Berkeley MIDS/W266/t5_dataset/BERT_X_Test_Predictions_1_to_5.csv", sep = ",", header=0)
# Merging BERT predictions and confidences for test set
test_BERT = pd.merge(test_BERT_preds,test_BERT_confidence,on = ["Unnamed: 0","Unnamed: 0"])

In [8]:
# Reading in CNN confidences and predictions for training set 2
train2_CNN = pd.read_csv("/content/drive/MyDrive/UC Berkeley MIDS/W266/t5_dataset/CNN_Confidence_x_train_stage_2_sampled_yelp_data.csv", sep = ",", header=0)
train2_CNN_preds = pd.read_csv("/content/drive/MyDrive/UC Berkeley MIDS/W266/t5_dataset/CNN_Predictions.csv", sep = ",", header=0)

In [9]:
# Reading in CNN confidences and predictions for test set
test_CNN = pd.read_csv("/content/drive/MyDrive/UC Berkeley MIDS/W266/t5_dataset/CNN_Confidence_x_test_sampled_yelp_data_NEW.csv", sep = ",", header=0)
test_CNN_preds = pd.read_csv("/content/drive/MyDrive/UC Berkeley MIDS/W266/t5_dataset/X_test_predictions_1_to_5_CNN.csv", sep = ",", header=0)

In [12]:
# Reading in predictions for T5 for training set 2 and test set
train2_t5 = pd.read_csv("/content/drive/MyDrive/UC Berkeley MIDS/W266/t5_dataset/t5_Predictions.csv", sep = ",", header=0),
test_results = pd.read_csv("/content/drive/MyDrive/UC Berkeley MIDS/W266/t5_dataset/t5_Predictions_test20k.csv", sep = ",", header=0)

In [11]:
# Combining all predictions and confidences for training set 2 as well as actual column
train2Results = pd.DataFrame({'BERT_1': train2_BERT['BERT_1'],
             'BERT_2': train2_BERT['BERT_2'],
             'BERT_3': train2_BERT['BERT_3'],
             'BERT_4': train2_BERT['BERT_4'],
             'BERT_5': train2_BERT['BERT_5'],
             'BERT_Predicted' : train2_BERT['0'],
             'CNN_1': train2_CNN['CNN1'],
             'CNN_2': train2_CNN['CNN2'],
             'CNN_3': train2_CNN['CNN3'],
             'CNN_4': train2_CNN['CNN4'],
             'CNN_5': train2_CNN['CNN5'],
             'CNN_Predicted' : train2_CNN_preds['CNN_predictions'],
             'T5 Predicted':train2_t5['Predicted'],
             'Actual' : train2_t5['Actual']
})

In [13]:
# Combining all predictions and confidences for test set as well as actual column
allTestResults = pd.DataFrame({'BERT_1': test_BERT['BERT_1'],
             'BERT_2': test_BERT['BERT_2'],
             'BERT_3': test_BERT['BERT_3'],
             'BERT_4': test_BERT['BERT_4'],
             'BERT_5': test_BERT['BERT_5'],
             'BERT_Predicted' : test_BERT['0'],
             'CNN_1': test_CNN['CNN1'],
             'CNN_2': test_CNN['CNN2'],
             'CNN_3': test_CNN['CNN3'],
             'CNN_4': test_CNN['CNN4'],
             'CNN_5': test_CNN['CNN5'],
             'CNN_Predicted' : test_CNN_preds['CNN_predictions'],
             'T5 Predicted':test_results['Predicted'],
             'Actual' : test_results['Actual']
})

In [14]:
# Rule based ensemble model with confidences
def ruleBasedEnsemble(df):
  count_allMatch = 0
  count_twoMatches_BERT_CNN_Picked = 0
  count_twoMatches_BERT_CNN_PickedT5 = 0
  count_twoMatches_BERT_T5_Picked = 0
  count_twoMatches_BERT_T5_PickedCNN = 0
  count_twoMatches_T5_CNN_Picked = 0
  count_twoMatches_T5_CNN_PickedBERT = 0
  count_oneMatch_BERT = 0
  count_oneMatch_CNN = 0
  count_oneMatch_T5 = 0
  for index, row in df.iterrows():
    # Get max confidence for bert and cnn
    bert_conf = max(row['BERT_1'],row['BERT_2'],row['BERT_3'],row['BERT_4'],row['BERT_5'])
    cnn_conf = max(row['CNN_1'],row['CNN_2'],row['CNN_3'],row['CNN_4'],row['CNN_5'])
    # Assumption: T5 confidence is 1

    # See if BERT, CNN, and T5 predictions all match
    if(row['BERT_Predicted']==row['CNN_Predicted']==row['T5 Predicted']):
      df.loc[index, 'Final Predicted']=row['BERT_Predicted']
      df.loc[index, 'Prediction used'] = "All three match"
      count_allMatch = count_allMatch+1

    # Else see if BERT and CNN predictions match
    elif((row['BERT_Predicted']==row['CNN_Predicted'])):
      if((bert_conf+cnn_conf)>=1):
        df.loc[index, 'Final Predicted']=row['BERT_Predicted']
        df.loc[index, 'Prediction used'] = "BERT & CNN"
        count_twoMatches_BERT_CNN_Picked = count_twoMatches_BERT_CNN_Picked+1
      else:
        # Revert to T5 prediction if BERT + CNN confidences are not greater than or equal to 1
        df.loc[index,'Final Predicted']=row['T5 Predicted']
        df.loc[index, 'Prediction used'] = "BERT & CNN Reverted to T5"
        count_twoMatches_BERT_CNN_PickedT5=count_twoMatches_BERT_CNN_PickedT5+1

    # Else see if BERT and T5 predictions match
    elif((row['BERT_Predicted']==row['T5 Predicted'])):
      if((bert_conf+1)/2 >=cnn_conf):
        df.loc[index, 'Final Predicted']=row['BERT_Predicted']
        df.loc[index, 'Prediction used'] = "BERT & T5"
        count_twoMatches_BERT_T5_Picked = count_twoMatches_BERT_T5_Picked+1
      else:
        # Revert to CNN if average confidence of BERT and T5 is less than CNN confidence
        df.loc[index, 'Final Predicted']=row['CNN_Predicted']
        df.loc[index, 'Prediction used'] = "BERT & T5 Reverted to CNN"
        count_twoMatches_BERT_T5_PickedCNN = count_twoMatches_BERT_T5_PickedCNN+1

    # Else see if CNN and T5 predictions match
    elif((row['CNN_Predicted']==row['T5 Predicted'])):
      # Revert to BERT if average confidence of CNN and T5 is less than BERT confidence
      if(((cnn_conf+1)/2 <=bert_conf)):
        df.loc[index, 'Final Predicted']=row['BERT_Predicted']
        df.loc[index, 'Prediction used'] = "CNN & T5 Reverted to BERT"
        count_twoMatches_T5_CNN_PickedBERT = count_twoMatches_T5_CNN_PickedBERT+1

      else:
        df.loc[index, 'Final Predicted']=row['CNN_Predicted']
        df.loc[index, 'Prediction used'] = "CNN & T5"
        count_twoMatches_T5_CNN_Picked = count_twoMatches_T5_CNN_Picked+1
        
    else:
      # Else use BERT prediction if BERT confidence is greater than or equal to 55%
      if(bert_conf>=0.55):
        df.loc[index,'Final Predicted']=row['BERT_Predicted']
        df.loc[index, 'Prediction used'] = "BERT"
        count_oneMatch_BERT=count_oneMatch_BERT+1

      # Else use CNN prediction if CNN confidence is greater than or equal to 45%
      elif(cnn_conf>=0.45):
        df.loc[index,'Final Predicted']=row['CNN_Predicted']
        df.loc[index, 'Prediction used'] = "CNN"
        count_oneMatch_CNN = count_oneMatch_CNN+1

      # Else use T5 prediction
      else:
        df.loc[index,'Final Predicted']=row['T5 Predicted']
        df.loc[index, 'Prediction used'] = "T5"
        count_oneMatch_T5=count_oneMatch_T5+1

  # Print out breakdown and ensure there are no unaccounted for predictions
  remainingRows = df.shape[0]-count_allMatch-count_twoMatches_BERT_CNN_Picked-count_twoMatches_BERT_CNN_PickedT5-count_twoMatches_BERT_T5_Picked-count_twoMatches_BERT_T5_PickedCNN-count_twoMatches_T5_CNN_Picked-count_twoMatches_T5_CNN_PickedBERT-count_oneMatch_BERT-count_oneMatch_CNN-count_oneMatch_T5
  print("All three match: ", count_allMatch)
  print("Two matches - BERT & CNN: ", count_twoMatches_BERT_CNN_Picked)
  print("Two matches - BERT & CNN Revert to T5: ", count_twoMatches_BERT_CNN_PickedT5)
  print("Two matches - BERT & T5: ", count_twoMatches_BERT_T5_Picked)
  print("Two matches - BERT & T5 Revert to CNN: ", count_twoMatches_BERT_T5_PickedCNN)
  print("Two matches - CNN & T5: ", count_twoMatches_T5_CNN_Picked)
  print("Two matches - CNN & T5 Revert to BERT: ", count_twoMatches_T5_CNN_PickedBERT)
  print("One match - BERT: ", count_oneMatch_BERT)
  print("One match - CNN: ", count_oneMatch_CNN)
  print("One match - T5: ", count_oneMatch_T5)
  print("Remaining: ", remainingRows)
  
  return df

In [15]:
# Run ensemble model on training 2 set
train2Results_ensemble = ruleBasedEnsemble(train2Results)
# Get accuracy
labels = train2Results_ensemble['Actual']
preds = train2Results_ensemble['Final Predicted']
matches = sum([1 if label == pred else 0 for label, pred in zip(labels, preds)])
print("Val accuracy: ", matches/80000)

All three match:  49451
Two matches - BERT & CNN:  10442
Two matches - BERT & CNN Revert to T5:  30
Two matches - BERT & T5:  8996
Two matches - BERT & T5 Revert to CNN:  12
Two matches - CNN & T5:  741
Two matches - CNN & T5 Revert to BERT:  6799
One match - BERT:  3485
One match - CNN:  23
One match - T5:  21
Remaining:  0
Val accuracy:  0.7225375


In [ ]:
# Results of fine-tuning ensemble model on training 2 set
# 71.01% train with BERT>= 0.85 and CNN >=0.5
# 71.15% train with BERT >= 0.8 and CNN >=0.5
# 71.2% train with BERT >= 0.75 and CNN >=0.5
# 71.23% train with BERT >= 0.7 and CNN >=0.5
# 71.24% train with BERT >= 0.65 and CNN >=0.5
# 71.25% train with BERT >= 0.6 and CNN >=0.5
# 71.25% train with BERT >= 0.55 and CNN >=0.5
# 71.25% train with BERT >= 0.55 and CNN >=0.55
# 71.25% train with BERT >= 0.55 and CNN >=0.45

# Ensemble model with reverts 
# 72.25%

In [16]:
# Looking at the groupings of the incorrect results
incorrectResults = train2Results_ensemble[train2Results_ensemble['Final Predicted']!=train2Results_ensemble['Actual']]
incorrectResults.groupby('Prediction used').size()

Prediction used
All three match              8962
BERT                         1850
BERT & CNN                   3733
BERT & CNN Reverted to T5      26
BERT & T5                    3724
BERT & T5 Reverted to CNN      11
CNN                            22
CNN & T5                      506
CNN & T5 Reverted to BERT    3347
T5                             16
dtype: int64

In [17]:
# Looking at validation accuracy for binned predictions (stars 1-2 binned and stars 4-5 binned) on train set 2
train2Results_ensemble['Final Predicted Binned'] = train2Results_ensemble['Final Predicted'].apply(lambda x: 1.0 if ((x == 1.0) | (x == 2.0)) else (5.0 if ((x == 4.0) | (x== 5.0)) else 3.0))
train2Results_ensemble['Actual Binned'] = train2Results_ensemble['Actual'].apply(lambda x: 1.0 if ((x == 1.0) | (x == 2.0)) else (5.0 if ((x == 4.0) | (x==5.0)) else 3.0))
labels = train2Results_ensemble['Actual Binned']
preds = train2Results_ensemble['Final Predicted Binned']
matches = sum([1 if label == pred else 0 for label, pred in zip(labels, preds)])
print("Val accuracy: ", matches/80000)

Val accuracy:  0.8936375


In [18]:
# Running ensemble model on test data
testResults_ensemble = ruleBasedEnsemble(allTestResults)
labels = testResults_ensemble['Actual']
preds = testResults_ensemble['Final Predicted']
matches = sum([1 if label == pred else 0 for label, pred in zip(labels, preds)])
print("Val accuracy: ", matches/20000)

All three match:  12200
Two matches - BERT & CNN:  2592
Two matches - BERT & CNN Revert to T5:  2
Two matches - BERT & T5:  2254
Two matches - BERT & T5 Revert to CNN:  0
Two matches - CNN & T5:  88
Two matches - CNN & T5 Revert to BERT:  1935
One match - BERT:  928
One match - CNN:  0
One match - T5:  1
Remaining:  0
Val accuracy:  0.72515


In [19]:
# Looking at validation accuracy for binned predictions (stars 1-2 binned and stars 4-5 binned) on test set
testResults_ensemble['Final Predicted Binned'] = testResults_ensemble['Final Predicted'].apply(lambda x: 1.0 if ((x == 1.0) | (x == 2.0)) else (5.0 if ((x == 4.0) | (x== 5.0)) else 3.0))
testResults_ensemble['Actual Binned'] = testResults_ensemble['Actual'].apply(lambda x: 1.0 if ((x == 1.0) | (x == 2.0)) else (5.0 if ((x == 4.0) | (x==5.0)) else 3.0))
labels = testResults_ensemble['Actual Binned']
preds = testResults_ensemble['Final Predicted Binned']
matches = sum([1 if label == pred else 0 for label, pred in zip(labels, preds)])
print("Val accuracy: ", matches/20000)

Val accuracy:  0.89505


In [20]:
# Looking at validation accuracy for binned predictions (stars 1-2 binned and stars 4-5 binned) on test set for baseline models
testResults_ensemble['BERT Predicted Binned'] = testResults_ensemble['BERT_Predicted'].apply(lambda x: 1.0 if ((x == 1.0) | (x == 2.0)) else (5.0 if ((x == 4.0) | (x== 5.0)) else 3.0))
preds = testResults_ensemble['BERT Predicted Binned']
matches = sum([1 if label == pred else 0 for label, pred in zip(labels, preds)])
print("BERT Binned Val accuracy: ", matches/20000)

testResults_ensemble['CNN Predicted Binned'] = testResults_ensemble['CNN_Predicted'].apply(lambda x: 1.0 if ((x == 1.0) | (x == 2.0)) else (5.0 if ((x == 4.0) | (x== 5.0)) else 3.0))
preds = testResults_ensemble['CNN Predicted Binned']
matches = sum([1 if label == pred else 0 for label, pred in zip(labels, preds)])
print("CNN Binned Val accuracy: ", matches/20000)

testResults_ensemble['T5 Predicted Binned'] = testResults_ensemble['T5 Predicted'].apply(lambda x: 1.0 if ((x == 1.0) | (x == 2.0)) else (5.0 if ((x == 4.0) | (x== 5.0)) else 3.0))
preds = testResults_ensemble['T5 Predicted Binned']
matches = sum([1 if label == pred else 0 for label, pred in zip(labels, preds)])
print("T5 Binned Val accuracy: ", matches/20000)

BERT Binned Val accuracy:  0.89525
CNN Binned Val accuracy:  0.8557
T5 Binned Val accuracy:  0.8468
